In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
breast_cancer_dataset = load_breast_cancer()

m_total = len(breast_cancer_dataset.target)

Y = breast_cancer_dataset.target.reshape(m_total, 1)
X = breast_cancer_dataset.data[:,np.r_[0:10]]
X_scaled = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.1)
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.T, y_test.T 

In [3]:
def init_params(layer_dims):
    params = {}
    np.random.seed(42)

    for i in range(1, len(layer_dims)):
        prev_layer_dims, current_layer_dims = layer_dims[i-1], layer_dims[i]
        params["W" + str(i)] = np.random.randn(current_layer_dims, prev_layer_dims) * 0.01
        params["b" + str(i)] = np.zeros((current_layer_dims, 1))
    return params

In [4]:
def relu_generator(base_leak):
    def relu(Z):
        return np.maximum(Z * base_leak, Z)
    return relu

def sigmoid(Z):
    return (1/(1 + np.exp(-Z)))

activation_index = {
    "sigmoid": sigmoid,
    "straigth_relu": relu_generator(0),
    "leaky_relu": relu_generator(0.04),
    "aggressive_relu": relu_generator(0.1)
}

def feed_forward_pass(prevA, W, b):
    Z = np.dot(W, prevA) + b
    assert(Z.shape == (W.shape[0], prevA.shape[1]))
    return Z

def feed_forward_activate(prevA, W, b, activation_func):
    Z = feed_forward_pass(prevA, W, b)
    A = activation_index[activation_func](Z)
    
    assert (A.shape == (W.shape[0], prevA.shape[1]))
    return A, Z, W, b

In [5]:
def compute_cost(Y, Af):
    m = Y.shape[1]
    logprobs = np.multiply(Y, np.log(Af)) + np.multiply((1 - Y), np.log(1 -Af))
    return -np.sum(logprobs)/m

In [6]:
def relu_deriv_generator(base_leak):
    def relu_deriv(Z):
        Z[Z>0] = 1
        Z[Z<0] = base_leak
        return Z
    return relu_deriv

def deriv_sigmoid(Z):
    return sigmoid(Z) * (1 - sigmoid(Z))

derivation_index = {
    "sigmoid": deriv_sigmoid,
    "straigth_relu": relu_deriv_generator(0),
    "leaky_relu": relu_deriv_generator(0.04),
    "aggressive_relu": relu_deriv_generator(0.1)
}

def backward_propagate(dA, prevA, layer_cache, activation_func, m):
    Z, W = layer_cache["Z"], layer_cache["W"]
    activation_derivation = derivation_index[activation_func](Z)
    dZ = np.multiply(dA, activation_derivation)

    dW = (1/m)*np.dot(dZ, prevA.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    prevdA = np.dot(W.T, dZ)
    
    return prevdA, dW, db

def update_params(params, grads, learning_rate, number_of_layers):
    for L in range(1, number_of_layers):
        params["W"+str(L)] -= learning_rate * grads["dW"+str(L)]
        params["b"+str(L)] -= learning_rate * grads["db"+str(L)]
    return params
      

In [9]:
def Deep_Model(X, Y, layer_dims, layer_types, learning_rate=0.0001, num_iterations=10000, debug=False):
    print('--------Model Init--------')
    ## Param initialisation
    params = init_params(layer_dims)
    costs = []

    ## Iteration initialisation
    for i in range(num_iterations):
        caches={}
        caches["0"] = {}
        caches["0"]["A"] = X
        
        A = X

        ## Forward propagation
        for L in range(1, len(layer_dims)):
            prevA = A
            A, Z, W, b = feed_forward_activate(
                prevA,
                params["W" + str(L)],
                params["b" + str(L)],
                layer_types[L]
            )
            caches[str(L)] = {}
            caches[str(L)]["W"] = W
            caches[str(L)]["b"] = b
            caches[str(L)]["A"] = A
            caches[str(L)]["Z"] = Z
            
        ## Compute cost
        cost = compute_cost(Y, A)
        costs.append(cost)
        
        if debug and i%1000 is 0: print(i, " : ", cost)
        
        ## Backward propagation
        grads = {}
        m = Y.shape[1]

        AL = caches[str(len(layer_dims)-1)]["A"]
        dAL = -(np.divide(Y, AL) + np.divide(1 - Y, 1 - AL))
        dA = dAL

        for L in list(reversed(range(1, len(layer_dims)))):
            activation_func = layer_types[L]
            layer_cache = caches[str(L)]
            prevA = caches[str(L-1)]["A"]

            prevdA, dW, db = backward_propagate(dA, prevA, layer_cache, activation_func, m)

            grads["dW" + str(L)] = dW
            grads["db" + str(L)] = db

            dA = prevdA
    
        ## Update parameters
        params = update_params(params, grads, learning_rate, len(layer_dims))
    
    print('--------Model trained--------')
    return params, costs

In [11]:
layer_dims = [
    X_train.shape[0],
    12,
    1
]

layer_types = [
    "input",
    "leaky_relu",
    "sigmoid"
]

params, costs = Deep_Model(X_train, y_train, layer_dims, layer_types, debug = True)

--------Model Init--------
0  :  0.693362836667
1000  :  0.686837654298
2000  :  0.680978122794
3000  :  0.675770340058
4000  :  0.67119987507
5000  :  0.667252230364
6000  :  0.663912126323
7000  :  0.661164407493
8000  :  0.658993942711
9000  :  0.657385947779
--------Model trained--------
